In [1]:
# https://marginaleffects.com/vignettes/gcomputation.html

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import statsmodels.formula.api as smf
import marginaleffects

In [3]:
cols = ["wt82_71", "qsmk", "sex", "race", "age", "education", "smokeintensity", "smokeyrs", "exercise", "active", "wt71"]

df = pd.read_csv("nhefs.csv", usecols=cols).dropna()
df.head()
df.shape

,qsmk,sex,age,race,education,wt71,wt82_71,smokeintensity,smokeyrs,active,exercise
0,0,0,42,1,1,79.04,-10.093960,30,29,0,2
1,0,0,36,0,2,58.63,2.604970,20,24,0,0
2,0,1,56,1,2,56.81,9.414486,20,26,0,2
3,0,0,68,1,1,59.42,4.990117,3,53,1,2
4,0,0,40,0,2,87.09,4.989251,20,19,1,1


(1566, 11)

## Estimate causal effect of smoking cessitation on dependent variable

In [4]:
# We want to estimate the effect of a binary treatment X on outcome Y, but there is a confounding variable W. 
# We can use standardization with the parametric g-formula to handle this. Roughly speaking, the procedure is as 
# follows:

# 1. Use the observed data to fit a regression model with Y as outcome, X as treatment, and W as control variable
#    (with perhaps some polynomials and/or interactions if there are multiple control variables).
# 2. Create a new dataset exactly identical to the original data, but where X = 1 in every row.
# 3. Create a new dataset exactly identical to the original data, but where X = 0 in every row.
# 4. Use the model from Step 1 to compute adjusted predictions in the two counterfactual datasets from Steps 2 and 3.
# 5. The quantity of interest is the difference between the means of adjusted predictions in the two counterfactual 
#    datasets.

# This is equivalent to computing an "Average Contrast", in which the value of moves from 0 to 1. 

In [5]:
formula = "wt82_71 ~ qsmk + sex + race + age + pow(age,2) + C(education) + \
    smokeintensity + pow(smokeintensity,2) + smokeyrs + \
    pow(smokeyrs,2) + C(exercise) + C(active) + wt71 + \
    pow(wt71,2) + qsmk*smokeintensity"

model = smf.ols(formula, data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                wt82_71   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     13.45
Date:                Sat, 01 Jun 2024   Prob (F-statistic):           1.47e-41
Time:                        19:19:02   Log-Likelihood:                -5328.6
No. Observations:                1566   AIC:                         1.070e+04
Df Residuals:                    1545   BIC:                         1.081e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [6]:
# qsmk is a binary measure of smoking cessation
cmp = marginaleffects.avg_comparisons(model, variables={"qsmk" : [0,1]}).to_pandas()
cmp

,term,contrast,estimate,std_error,statistic,p_value,s_value,conf_low,conf_high
0,qsmk,mean(True) - mean(False),3.517374,0.440282,7.98892,1.332268e-15,49.415037,2.654438,4.38031


In [7]:
# Same calculation
model.predict(df.assign(qsmk=1)).mean() - model.predict(df.assign(qsmk=0)).mean()

3.5173742008854774

## More details

In [8]:
# average predicted outcome in the original data
p = marginaleffects.predictions(model).to_pandas()
p.head()
p["estimate"].mean()

,rowid,estimate,std_error,statistic,p_value,s_value,conf_low,conf_high,qsmk,sex,age,race,education,wt71,wt82_71,smokeintensity,smokeyrs,active,exercise
0,0,4.143747,0.791779,5.233462,1.663638e-07,22.519155,2.591888,5.695606,0,0,42,1,1,79.04,-10.093960,30,29,0,2
1,1,6.328048,0.687164,9.208932,0.000000e+00,inf,4.981231,7.674865,0,0,36,0,2,58.63,2.604970,20,24,0,0
2,2,1.980843,0.828724,2.390231,1.683779e-02,5.892153,0.356573,3.605113,0,1,56,1,2,56.81,9.414486,20,26,0,2
3,3,-4.064697,1.064558,-3.818200,1.344288e-04,12.860871,-6.151194,-1.978201,0,0,68,1,1,59.42,4.990117,3,53,1,2
4,4,2.281337,0.655139,3.482221,4.972728e-04,10.973675,0.997289,3.565385,0,0,40,0,2,87.09,4.989251,20,19,1,1


2.6382997865654874

In [9]:
# average predicted outcome in the two counterfactual datasets
newdata = marginaleffects.datagrid(qsmk=[0,1], grid_type="counterfactual")
p = marginaleffects.predictions(model, newdata=newdata).to_pandas()
p.groupby("qsmk")["estimate"].mean()

qsmk
0    1.756213
1    5.273587
Name: estimate, dtype: float64

In [10]:
# Calculation by hand
# also works, but no standard error or confidence interval
model.predict(df).mean()
model.predict(df.assign(qsmk=0)).mean()
model.predict(df.assign(qsmk=1)).mean()
model.predict(df.assign(qsmk=1)).mean() - model.predict(df.assign(qsmk=0)).mean()

2.6382997865654874

1.7562131154684735

5.273587316353951

3.5173742008854774